In [ ]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Modeling Trees as Truncated Cones using Convex Optmization

The purpose of this notebook is to fit the truncated cone model introduced in the previous notebook using convex optimization. 

## Intro

It is a common practice in forestry to estimate tree volume from measurements of tree diameter (DBH) and height. This topic is of key interest to researchers and industry. In forestry, it is economically valuable to have a reliable estimate of usable timber in a stand of trees that might take years to mature. In researching climate change, it is valuable to know how much carbon trees can sequester, which is related to the volume of the tree.  

The purpose of this notebook is to present a model of tree volume based on the geometry of a truncated cone. For more on the standard regression approaches to the problem, see the previous notebook. The data used will be the Trees dataset built-in to the R programming language.

All code can be found on: https://github.com/jh-206/Tree-Volume

<img src="images/tcone.png" alt="alt text" style="width: 300px;"/>

## Model Formulation

The volume observations in the data is sometimes referred to as "merchantable volume". It is found by removing branches from trunk of the tree, and the final volume value is directly related to the amount of usable timber in the trunk of the tree. For these reasons, the trunk looks like a big truncated cone. A truncated cone is defined by a lower radius and an upper radius.

$$
V = \frac{1}{3}\pi (r_1^2+r_1r_2+r_2^2)h
$$

Where $V$ is the observed volume of the tree and $h$ is the observed height. Then, $r_1$ and $r_2$ are the radii of the base and top of the cone, respectively.

We will consider the upper radius to be a fraction of the lower radius. So, we introduce a parameter $\alpha$ where $r_2 = \alpha r_1, \text{ for }\alpha \in (0,1)$. We will use a grid search to find an alpha that minimizes the fitted sum of squares. Let the lower tree radius be $r$ and the upper be $r_2$. Since $r_2 = \alpha r$, using $\hat V_i$ to represent the estimated volume of tree $i$, we can write the volume as:

$$
\hat V_i = \frac{1}{3}\cdot \pi h_i r_i^2(1+\alpha+\alpha^2), \;\alpha\in(0,1), i=1, ..., N
$$

Where $V_i$ is the observed volume of the $i$th tree, where $i$ equals $1, ..., N$ where $N$ is the total number of observations.

Formulating this as an optimization problem, we seek to minimize the sum of squares of the residuals. The observed values are radius ($r_i$), height ($h_i$), and volume ($V_i$), for $i=1, ..., N$. Here, $N$ is the total number of observations (31 for the trees dataset).

We want to minimize the sum of squared residuals, so our loss function is:

$$
\begin{align*}
    \min &\sum_{i=1}^N \left(\pi\frac{h_i}{3} r_i^2(1+\alpha+\alpha^2) - V_i\right)^2\\
    \text{s.t. }&\alpha\in[0,1]
\end{align*}
$$

To represent this in matrix notation, Let $V=\begin{pmatrix}V_1 \\ V_2 \\ \vdots \\ V_N\end{pmatrix}$ and $hr^2 = \begin{pmatrix}h_1r_1^2 \\ h_2r_2^2 \\ \vdots \\ h_Nr_N^2\end{pmatrix}$

Putting this all together, we have:

$$
\begin{align*}
    \min &\|\pi\frac{h}{3} r^2(1+\alpha+\alpha^2)-V\|_2^2 \\
    \text{s.t. }&0\leq\alpha\leq 1
\end{align*}
$$

Where solutions are scalar values $\alpha\in \mathbb R$, with $\alpha\in [0,1]$

### Code-Friendly Formulation

We seek to write the minimation problem in the form $\min (Ax-b)$ to play nice with `cvxpy`.

First, since $1+\alpha+\alpha^2$ is one-to-one over $[0,1]$, we can define $a=1+\alpha+\alpha^2$ and then use the optimal value of $a$ to uniquely solve for $\alpha$. If $\alpha\in[0,1]$, we know that $1+\alpha+\alpha^2$ is in the interval $[1, 3]$.

So we convert the constraint from the previous problem to $a\in[1,3]$

$$
\begin{align*}
    \min &\|\pi\frac{h}{3} r^2 a - V\|_2^2 \\
    \text{s.t. }&1\leq a \leq 3
\end{align*}
$$

## Read Data and Set Parameters

In [ ]:
trees = pd.read_csv("trees.csv")
trees.head()

In [ ]:
r = (trees.d/2).to_numpy() # DBH Divided by height
h = trees.h.to_numpy()
V = trees.v.to_numpy()

## Optimization

In [ ]:
np.random.seed(6596) # NLP CU Denver Course number used as seed

a = cp.Variable(1, pos=True) # alpha parameter, target of interest
objective=cp.sum_squares(h/3*np.pi*r**2 * a - V) # Objective Function
constraints = [a>= 1, a<=3]
prob = cp.Problem(cp.Minimize(objective), constraints)
prob.solve()

In [ ]:
a.value

### Final Optimization Solution

Using the optmized value of $a$, we solve for the unique value of $\alpha$ by first setting up a quadratic root problem:

$$
1+\alpha+\alpha^2 = a
$$

Then, we use the QUADRATIC FORMULA(!!) and ignore the negative root that would correspond to an $\alpha$ outside of $[0,1]$.

$$
\frac{-1\pm\sqrt{1-4*1*(1-a)}}{2}
$$

We calculate with code below:

In [ ]:
print((-1-np.sqrt(1-4*(1-a.value)))/2)
print((-1+np.sqrt(1-4*(1-a.value)))/2)

Therefore, the final estimate of $\alpha$ is $0.13995219$, or roughly $0.14$. Connecting this back to the physical problem, that means that if we represent a tree trunk as a truncated cone with a known lower radius $r$, the volume of the tree will be approximated best by using an upper radius of $0.14*r$.

## Error and Uncertainty Analysis

To estimate the error associated with the model, and additionally to produce a confidence interval for the alpha parameter, we perform the following cross validation procedure: for 1,000 replications, split the data into an 80/20 percentage split, find the optimal parameter for the training set, and calculate the error when using that optimal parameter to predict new data. (Note: this is a form of cross-validation sometimes called Monte Carlo CV. There are more elegant coding approaches to this, but I think this is good for clearly demonstrating the technique.)

Koirala etal 2017 arrived at the following model specification after testing various formulations:
$$
V = \beta_0+\beta_1\cdot d + \beta_2\cdot d^2\cdot h 
$$

The model therefore has 3 uncertain parameters, and a 4th if you add estimating the random error variance that is typical in linear regression models. This model will be compared to the truncated cone model.

In [ ]:
nreps = 1000
alphas = np.zeros(nreps) # initialize array of parameters
rmses = np.zeros(nreps) # initialize array of RMSE errors
rmses_lm = np.zeros(nreps) # initialize array of RMSE errors
np.random.seed(123)

for i in range(0, nreps):
    # print("~"*40)
    # print(f"Iteration {i}")
    # print(f"Seed: {}")
    # Split Data
    np.random.seed()
    inds = np.random.choice([0, 1], size=len(trees), p=[0.8, 0.2]) # generate labels for train/test
    train = trees[inds == 0]
    test = trees[inds == 1]
    # Find optimal param
    r = (train.d/2).to_numpy() # DBH Divided by height
    h = train.h.to_numpy()
    V = train.v.to_numpy()
    a = cp.Variable(1, pos=True) # alpha parameter, target of interest
    np.random.seed(6596) # NLP CU Denver Course number used as seed
    objective=cp.sum_squares(h/3*np.pi*r**2 * a - V) # Objective Function
    constraints = [a>= 1, a<=3]
    prob = cp.Problem(cp.Minimize(objective), constraints)
    temp = prob.solve()
    alphas[i] = (-1+np.sqrt(1-4*(1-a.value[0])))/2

    # LM Model
    X = train[["d", "h"]].copy()
    X=X.assign(d2h = X['d']**2 * X['h'])
    X["const"] = np.ones(X.shape[0])
    lm = sm.OLS(train.v, X).fit()
    
    X_test = test[["d", "h"]].copy()
    X_test=X_test.assign(d2h = X_test['d']**2 * X_test['h'])
    X_test["const"] = np.ones(X_test.shape[0])
    
    # Predict and calculate error
    preds = test['h']/3*np.pi*(test['d']/2)**2 * a.value[0]
    rmses[i] = np.sqrt(np.sum((preds-test.v)**2))

    preds2 = lm.predict(X_test)
    rmses_lm[i] = np.sqrt(np.sum((preds2-test.v)**2))

In [ ]:
print(f"Mean Alpha: {alphas.mean()}")
print(f"95% Empirical Interval: {np.percentile(alphas, 2.5), np.percentile(alphas, 97.5)}")

In [ ]:
plt.hist(alphas)
plt.xlabel("Alpha")
plt.ylabel("Frequency")
plt.axvline(alphas.mean(), color='r', linestyle='--', linewidth=2)
plt.text(alphas.mean(), plt.ylim()[1]*.7, r'Mean $\alpha$', rotation=90, color='r', verticalalignment='bottom', horizontalalignment='right')

plt.axvline(np.percentile(alphas, 2.5), color='r', linestyle='--', linewidth=2, alpha=.6)
plt.text(np.percentile(alphas, 2.5), plt.ylim()[1]*.7, r'2.5 percentile', rotation=90, alpha=.6, color='r', verticalalignment='bottom', horizontalalignment='right')

plt.axvline(np.percentile(alphas, 97.5), color='r', linestyle='--', linewidth=2, alpha=.6)
plt.text(np.percentile(alphas, 97.5), plt.ylim()[1]*.7, r'97.5 percentile', rotation=90, color='r', alpha=.6, verticalalignment='bottom', horizontalalignment='right')

In [ ]:
pd.DataFrame({
    'Model' : ["T. Cone", "Lin Reg"],
    'Mean Test RMSE' : np.round([rmses.mean(), rmses_lm.mean()], 3),
    'Low (2.5%)' : np.round([np.percentile(rmses, 2.5), np.percentile(rmses_lm, 2.5)], 3),
    'High (97.5%)' : np.round([np.percentile(rmses, 97.5), np.percentile(rmses_lm, 97.5)], 3)
})

## Adding Species

In the absence of data from multiple species, I will generate random species labels for my existing dataset and use that for modeling. In theory, there should be zero effect from species, since the species labels are generated randomly and have no relation to the outcome variable of volume.

In [ ]:
# Randomly generate species
np.random.seed(6596) # NLP CU Denver Course number used as seed
trees['species'] = np.random.randint(2, size=len(trees))
trees.head()

We introduce a simple additive effect from species, denoted $\beta$. So we modify the original volume formula by adding a parameter by adding $\beta$ times the species label. Suppose $S_i$ is the species of observation $i$. Then the volume formula of the truncated cone with an additive constant effect from species, we get:

$$
V_i = \frac{1}{3}\cdot \pi h_i r_i^2(1+\alpha+\alpha^2)+\beta S_i, \;\alpha\in(0,1), i = 1, ..., N
$$

$$
\begin{align*}
    \min &\|\frac{h}{3}\pi r^2 a + \beta S - V\|_2^2 \\
    \text{s.t. }&1\leq a \leq 3
\end{align*}
$$

We solve this problem with `cvxpy`. We again transform $a$ back into the parameter $\alpha$.

In [ ]:
r = (trees.d/2).to_numpy() # DBH Divided by height
h = trees.h.to_numpy()
V = trees.v.to_numpy()
S = trees.species.to_numpy()
a = cp.Variable(1, pos=True) # alpha parameter, target of interest
b = cp.Variable(1) # beta parameter, additivie species effect
np.random.seed(6596) # NLP CU Denver Course number used as seed

objective=cp.sum_squares(h/3*np.pi*r**2 * a+b*S - V) # Objective Function
constraints = [a>= 1, a<=3]
prob = cp.Problem(cp.Minimize(objective), constraints)
prob.solve()

In [ ]:
print(f"Alpha Value: {(-1+np.sqrt(1-4*(1-a.value)))/2}")
print(f"Beta value: {b.value}")

We expected the $\beta$ parameter value to be zero, since there is no actual effect from species. But this effect is small relative to the average observed volume of a tree and the average fitted volume from the truncated cone model.

In [ ]:
print(f"Average Observed Volume: {np.round(trees.v.mean(), 3)}")
print(f"Optimized Species Effect: {np.round(b.value[0], 3)}")
print(f"Species Effect, Percent of Average Volume: {np.round(100*b.value[0]/trees.v.mean(), 3)}")

So even though the parameter $\beta$ is nonzero, the optmized value represents less than 1% of the average tree volume.

As a further check, we repeat the randomly species labeling 1,000 times and examine the resulting $\beta$ values.

In [ ]:
np.random.seed(6596) # NLP CU Denver Course number used as seed
nreps = 1000
betas = np.zeros(nreps) # initialize vector of beta valuees
for i in range(0, nreps):
    trees['species'] = np.random.randint(2, size=len(trees)) # randomly assign species labels
    S = trees.species.to_numpy() # re-extract species label to array
    a = cp.Variable(1, pos=True) # re-declare beta parameter
    b = cp.Variable(1) # re-declare beta parameter
    objective=cp.sum_squares(h/3*np.pi*r**2 * a+b*S - V) # Objective Function
    constraints = [a>= 1, a<=3]
    # Solve problem
    prob = cp.Problem(cp.Minimize(objective), constraints)
    prob.solve()
    # Extract beta value for this iteration
    betas[i]=b.value[0]

In [ ]:
print(f"Average Beta Value: {np.round(betas.mean(), 3)}")
print(f"Average Beta Value as Percent of Mean Volume: {np.round(100*betas.mean() / trees.v.mean(), 3)}")

In [ ]:
plt.hist(betas)
plt.title("Distribution of Beta Parameter")
plt.xlabel("Beta")
plt.ylabel("Frequency")

The resulting distribution of $\beta$ values is centered on zero and has a roughly normal distribution. This matches what we would expect theoretically based on the Central Limit Theorem. 

## Discussion and Further Research

This report shows how to solve a geometrically formulated estimate of tree volume, given the typical forestry field measurements of tree diameter and height. This formulation of a volume model has advantages over standard statistical approaches in a few ways. First, the basic truncated cone model has only one uncertain parameter, while typical linear regression methods would use up degrees of freedom estimating typically 4 or more paramters (one each for height, diameter, random error, and a constant mean effect). Second, the geometrically inspired model gives potentially more physical insight. The truncated cone formulation can be used to get a natural estimate of surface area of the tree, for example. The surface area of a tree might be a useful value to estimate, if for instance the logs were to be painted or wrapped in some material and you wanted an estimate of how much you would need. A standard linear regression model would provide zero estimate of surface area. 

This report shows how an effect from species could be added, given more data for different tree species. Numerical experimentation above shows that if species labels are random, and thus have no relationship with volume, the resulting estimates of the additive effect from species vary about the expected value of zero with a roughly normal distribution.

This project could be developed and extended to a scientific contribution to forestry modeling. First, we could collect more data, preferrably the data used to develop models published in the academic literature and cited in the references of this report. Next, we could compare the truncated cone volume model to the various other models in the literature. Preliminary investigation shows that the truncated cone volume model fits this data of 31 cherry trees more accurately and with less uncertainty of relevant parameters.

## References

Koirala, Anil & Kizha, Anil Raj & Baral, Srijana. (2017). Modeling Height-Diameter Relationship and Volume of Teak (Tectona grandis L. F.) in Central Lowlands of Nepal. Journal of Tropical Forestry and Environment. 07. 28-42. 10.31357/jtfe.v7i1.3020. 

Birteeb, Peter & Ajit, & Varghese, Cini & Jaggi, Seema. (2020). Development and comparative diagnosis of conventional (linear/nonlinear) and artificial intelligence techniques-based predictive models for estimating timber volume of Tectona grandis. International Journal of Ecology and Environmental Sciences. 2. 1-11. 

https://stats.libretexts.org/Bookshelves/Probability_Theory/Book%3A_Introductory_Probability_(Grinstead_and_Snell)/09%3A_Central_Limit_Theorem/9.01%3A_Central_Limit_Theorem_for_Bernoulli_Trials#:~:text=Central%20Limit%20Theorem%20for,%CF%95(x)dx%20.&text=This%20theorem%20can%20be%20proved,k)%20given%20in%20Theorem%209.1.